# Loading Data
In lessons `02` and `03` we explored what data makes up an Evaluation dataset and how it can be accessed using the Evaluation table sub-classes.  In lesson `02` we cloned a complete datasets from AWS S3 to explore. In this lesson we are going to explore how your data can be added to a new or existing Evaluation dataset.

Similar to previous lessons we will start by creating a Evaluation class instance that references a directory where the data will be stored, and then clone the Evalation template as a starting point.

In [1]:
from teehr import Evaluation
from pathlib import Path
import shutil

# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "04_loading_data")
shutil.rmtree(test_eval_dir, ignore_errors=True)

# Create an Evaluation object and create the directory
ev = Evaluation(dir_path=test_eval_dir, create_dir=True)

# Clone the template
ev.clone_template()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/08 09:01:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
